In [3]:
import numpy as np
import scipy.stats as stats
from hansolo import *

Nb = 100
expert = "EWA"  # "EWA" or "PWA" or "Multilin"
learning = "blocs" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement


M = 2502
M_obj = 500 #size of testing set
n_epochs = int(M/9) #number of epochs
dt = 0.002
N = 1000
T = N * dt

freq = 100  # firing rate input neurons +
freq2 = 150

alpha = np.array([100 * dt, 0])  # spontaneous rates of output neurons

p = freq * dt  # spiking proba input neurons
p2 = freq2 * dt

K_input = 12  # nb experts
n_input = 6  # nb input neurons
K_output = 2  # nb output neurons


obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
]
# blue circle blue square blue triangle red circle red square red triangle gray circle gray square gray triangle



answers = np.zeros((Nb, n_epochs, M_obj))  # 0 if wrong, 1 if correct

W_output = np.load(f'saves/{expert}_{learning}_W_HAN.npy')
W_output=W_output[:,:,9*np.arange(int(M/9)),:] #weights after each epoch



for nb in range(Nb):
    #creation of the list of objects (testing set)
    L=list_objects_random(obj,M_obj)

    F_input = np.zeros((n_input, n_epochs, M_obj))
    F_output=np.zeros((K_output, n_epochs, M_obj))

    
    for m_obj in range(M_obj):

        obj_cur = L[m_obj]
        cur_act = np.where(obj_cur == 1)[0]
        input_neurons=np.zeros((n_input, n_epochs, N))
        input_neurons[cur_act, :,:] = Poisson(freq, T, (cur_act.size, n_epochs, N))

        output_neurons=np.zeros((K_output, n_epochs, N))
        for m in range(n_epochs):

            # simulation of the output neurons
            
            for j in range(K_output):
                probas = alpha[j] + np.sum(
                    (
                        W_output[j, :n_input, m, nb][:, np.newaxis]
                        - W_output[j, n_input:, m, nb][:, np.newaxis]
                    )
                    * input_neurons[:,m,:],
                    axis=0,
                )
                clipped_probas = np.clip(probas, 0, 1)
                output_neurons[j, m, :] = stats.bernoulli.rvs(clipped_probas)

        # firing rates
        F_input[:, :, m_obj] = np.sum(input_neurons, axis=2) / T
        F_output[:, :, m_obj] = np.sum(output_neurons, axis=2) / T

        for m in range(n_epochs):
            if (in_A(obj_cur) and F_output[0, m, m_obj] > F_output[1, m, m_obj]) or (
                in_B(obj_cur) and F_output[1, m, m_obj] > F_output[0, m, m_obj]
            ):
                answers[nb, m, m_obj] = 1
        if m_obj%100==0:
            print(nb,m_obj)


np.save(f"{expert}_{learning}_HAN_test",answers)
#np.save(f"{expert}_{learning}_W_HAN", W_output)
#np.save(f"{expert}100F_in", F_output)


0 0
0 100
0 200
0 300
0 400
1 0
1 100
1 200
1 300
1 400
2 0
2 100
2 200
2 300
2 400
3 0
3 100
3 200
3 300
3 400
4 0
4 100
4 200
4 300
4 400
5 0
5 100
5 200
5 300
5 400
6 0
6 100
6 200
6 300
6 400
7 0
7 100
7 200
7 300
7 400
8 0
8 100
8 200
8 300
8 400
9 0
9 100
9 200
9 300
9 400
10 0
10 100
10 200
10 300
10 400
11 0
11 100
11 200
11 300
11 400
12 0
12 100
12 200
12 300
12 400
13 0
13 100
13 200
13 300
13 400
14 0
14 100
14 200
14 300
14 400
15 0
15 100
15 200
15 300
15 400
16 0
16 100
16 200
16 300
16 400
17 0
17 100
17 200
17 300
17 400
18 0
18 100
18 200
18 300
18 400
19 0
19 100
19 200
19 300
19 400
20 0
20 100
20 200
20 300
20 400
21 0
21 100
21 200
21 300
21 400
22 0
22 100
22 200
22 300
22 400
23 0
23 100
23 200
23 300
23 400
24 0
24 100
24 200
24 300
24 400
25 0
25 100
25 200
25 300
25 400
26 0
26 100
26 200
26 300
26 400
27 0
27 100
27 200
27 300
27 400
28 0
28 100
28 200
28 300
28 400
29 0
29 100
29 200
29 300
29 400
30 0
30 100
30 200
30 300
30 400
31 0
31 100
31 200
31 300
3